# Recommender Systems 2021/22

### Practice - Hyperparameter optimization with Skopt

### Hyperparameter optimization is essential to achieve the best recommendation quality!!

## How does it work
* Split the data in training, validation and test. Ensure that the distribution of the three sets is similar (e.g., the validation data you use should be split with the same approach used for the test data; if you know the test data contains cold items or users, ensure the validation data does too...)
* Choose a recommender model, identify its hyperparameters and choose a value range and distribution for each.
* Select a hyperparameter configuration and fit your model using the training data. Evaluate it using the validation data.
* Repeat the previous step by exploring many possible hyperparameter configurations. Several exploration strategies are possible.
* Select the hyperparameter configuration with the best recommendation quality on the validation data. Use that configuration to fit the model on the union of training and validation data. Evaluate the final model on the test data and report that result (or submit to the course challenge).

Using directly or indirectly any information on the composition of the test data at any stage of the training or optimization process will result in *information leakage* and cause you to overestimate the quality of your model.



## Hyperparameter optimization strategies

### Grid-search
For each hyperparameter define a list of possible values, then explore *all* possible combinations. For example, in a KNN using tversky similarity you could choose these values:
* number of neighbors: [10, 50, 100, 150, 200, 250]
* shrink term: [0, 10, 20, 50, 100]
* alpha: [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5]
* beta: [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5]

You end up with *1920* possible hyperparameter configurations. Furthermore, you cannot easily fine-tune. Say that you found the optimal number of neighbors is 200, but in that area of the search space you were using a step of 50. What if there is an even better result at 215? You have to define a *new* hyperparameter range around the optimal one you found and use a smaller step, e.g., [180, 190, 200, 210, 220] and so on. 

Overall, grid-search is very sensitive to the range and distribution you choose, very rigid, and often unpractically slow due to how the number of cases grows combinatorially. It has been known for more than 20 years that it is generally a bad idea.

### Random-search
In a random search you define the range and distribution of possible values, and then you pick at random from it each hyperparameter configuration you explore.
For example, in a KNN using tversky similarity you could choose these values:
* number of neighbors: uniform random from 10 to 500
* shrink term: uniform random from 0 to 500
* alpha: uniform random from 0.1 to 1.5
* beta: uniform random from 0.1 to 1.5

It is very parallelizable and effective (definitely better than Grid-search).


### Bayesian-search
More advanced strategy that uses a gaussian process to try to model the interdependencies between hyperparameters based on how they affect the result. It contains a part of random search where the search space is explored, then a second part that uses the gaussian process to choose the next hyperparameter configuration. It combines exploration with exploitation. 

Bayesian-search is less parallelizable (the gaussian process is sequential) but has a good exploration-exploitation tradeoff. It is the strategy used in our research activity.


### Other strategies
There is no shortage of hyperparameter optimization strategies, for example the Tree-Structured Parzen Estimator, as well as techniques that use previous optimization runs to attempt transfer learning between models and datasets. All those are beyond the scope of the course.



### In the course repository you will find a BayesianSearch object in the HyperparameterTuning folder. That is a simple wrapper of another library and its purpose is to provide a very simple way to tune some of the most common hyperparameters. 




## Hyperparameter sensitivity

One of the most important problems you encounter when doing hyperparameter tuning is how to select the range and distribution of the hyperparameters. There is no universal rule and some of those decisions are based on experience. Generally the suggestions we can give you are:
* Keep the search space on the larger side, better have more room to manouver than less.
* Hyperparameters like the number of neighbors, the shrink term, the number of latent dimensions of matrix factorization models etc. work well with a uniform random distribution
* Hyperparameters like the learning rate and l1 l2 regularizations work better with a log-uniform distribution as the training tends to be affected by the orders of magnitude rather than the absolute values. You need to be able to explore maybe from 10^-9 to 10^-3, but is less important to choose the specific value.

### In the course repository you will find a script called run_hyperparameter_search that contains a list of commonly used hyperparameters with the corresponding range and distribution.

### Hint: If you see that the optimization yelds a hyperparameter that has a value at either end of the range (min or max) this may indicate that in that scenarion you need to expand the search space.



## Early-stopping

A special hyperparameter is the number of epochs you should train a machine learning model for.
It is possible to put this number as a hyperparameter, but that is not a super-effective strategy. A common strategy is *early-stopping*. It works as follows:
* Select a maximum number of epochs, say 500
* Train the model for a certain number of epochs, say 5
* Evaluate the recommendation quality of the model on the validation data. Create a clone of the model.
* Continue training and evaluate the model periodically. Every time you find a better recommendation quality update the model clone, which will then represent the "best" model you have.
* If the recommendation quality does not improve for a certain number of consecutive validation steps, say 5, stop the training. If you reach the maximum number of epochs, stop the training.
* Use the "best" model clone to generate the recommendations on the validation data.
* When you train the model on the union of training and validation data, use the optimal number of epochs selected by the early-stopping at the previous step.

Usually early-stopping allows to save a lot of computational time and fine-tune the optimal number of epochs, unless the validation step takes a very large amount of time. In those cases, it can be better to either use early-stopping on the algorithm loss function (which is however a different problem then recommendation and does not guarantee you that the best absolute loss will correspond to the model with the best recommendation quality) or just select the number of epochs as any other hyperparameter.


# Example on the course repository

In [1]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens1MReader import Movielens1MReader

data_reader = Movielens1MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()
ICM_all = data_loaded.get_ICM_from_name("ICM_genres")

Movielens1M: Verifying data consistency...
Movielens1M: Verifying data consistency... Passed!
DataReader: current dataset is: <class 'Data_manager.Dataset.Dataset'>
	Number of items: 3883
	Number of users: 6040
	Number of interactions in URM_all: 1000209
	Value range in URM_all: 1.00-5.00
	Interaction density: 4.26E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.66E+02
		 Max: 2.31E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 2.58E+02
		 Max: 3.43E+03
	Gini Index: 0.53

	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 18, feature occurrences: 6408, density 9.17E-02
	ICM name: ICM_year, Value range: 1.92E+03 / 2.00E+03, Num features: 1, feature occurrences: 3883, density 1.00E+00




### How do we perform hyperparameter optimization?
* Split the data in three *disjoint* sets: training, validation and testing data
* Define a set of hyperparameters with the range and distribution
* Explore hyperparameter space and select those with the best recommendation quality on the *validation* data (including the number of epochs for ML algorithms)
* Given the best hyperparameters, fit the model again using the union of training and validation data.
* Evaluate this last model on the testing data.

### Step 1: Split the data and create the evaluator objects

In [2]:
from Evaluation.Evaluator import EvaluatorHoldout

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 6024 ( 0.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 6036 ( 0.1%) Users that have less than 1 test interactions


### Step 2: Define hyperparameter set for the desired model, in this case ItemKNN

In [3]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(5, 1000),
    "shrink": Integer(0, 1000),
    "similarity": Categorical(["cosine"]),
    "normalize": Categorical([True, False]),
}

### Step 3: Create SearchBayesianSkopt object, providing the desired recommender class and evaluator objects

In [4]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = ItemKNNCFRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

### Step 4: Provide the data needed to create an instance of the model, one trained only on URM_train, the other on URM_train_validation

In [5]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

In [6]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

### Step 5: Create a result folder and select the number of cases (50 with 30% random is a good number)

In [7]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 10  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

### Step 5: Run!

In [8]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 150, 'shrink': 314, 'similarity': 'cosine', 'normalize': True}
ItemKNNCFRecommender: URM Detected 238 ( 6.1%) items with no interactions.
Similarity column 3883 (100.0%), 5210.63 column/sec. Elapsed time 0.75 sec
EvaluatorHoldout: Processed 6024 (100.0%) in 5.41 sec. Users per second: 1114
SearchBayesianSkopt: New best config found. Config 0: {'topK': 150, 'shrink': 314, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2026228, PRECISION_RECALL_MIN_DEN: 0.2288963, RECALL: 0.1204475, MAP: 0.1059938, MAP_MIN_DEN: 0.1170157, MRR: 0.4239689, NDCG: 0.1923506, F1: 0.1510842, HIT_RATE: 0.7981408, ARHR_ALL_HITS: 0.6578372, NOVELTY: 0.0241524, AVERAGE_POPULARITY: 0.4900907, DIVERSITY_MEAN_INTER_LIST: 0.9322565, DIVERSITY_HERFINDAHL: 0.9932102, COVERAGE_ITEM: 0.2722122, COVERAGE_ITEM_CORRECT: 0.1738347, COVERAGE_USER: 0.9973510, COVERAGE_USER_CORRECT: 0.7960265, DIVERSITY

Similarity column 3883 (100.0%), 2689.85 column/sec. Elapsed time 1.44 sec
EvaluatorHoldout: Processed 6024 (100.0%) in 9.61 sec. Users per second: 627
SearchBayesianSkopt: Config 4 is suboptimal. Config: {'topK': 783, 'shrink': 0, 'similarity': 'cosine', 'normalize': False} - results: PRECISION: 0.1420983, PRECISION_RECALL_MIN_DEN: 0.1581309, RECALL: 0.0767443, MAP: 0.0734144, MAP_MIN_DEN: 0.0803848, MRR: 0.3458051, NDCG: 0.1431956, F1: 0.0996628, HIT_RATE: 0.6636786, ARHR_ALL_HITS: 0.4954128, NOVELTY: 0.0221433, AVERAGE_POPULARITY: 0.7557058, DIVERSITY_MEAN_INTER_LIST: 0.5132387, DIVERSITY_HERFINDAHL: 0.9513154, COVERAGE_ITEM: 0.0159670, COVERAGE_ITEM_CORRECT: 0.0121040, COVERAGE_USER: 0.9973510, COVERAGE_USER_CORRECT: 0.6619205, DIVERSITY_GINI: 0.0050268, SHANNON_ENTROPY: 4.5819229, RATIO_DIVERSITY_HERFINDAHL: 0.9520925, RATIO_DIVERSITY_GINI: 0.0143769, RATIO_SHANNON_ENTROPY: 0.4237471, RATIO_AVERAGE_POPULARITY: 2.7359250, RATIO_NOVELTY: 0.0750299, 

Iteration No: 5 ended. Search fi

Similarity column 3883 (100.0%), 1733.86 column/sec. Elapsed time 2.24 sec
EvaluatorHoldout: Processed 6036 (100.0%) in 8.20 sec. Users per second: 736
SearchBayesianSkopt: Best config evaluated with evaluator_test with constructor data for final test. Config: {'topK': 300, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True} - results:
CUTOFF: 10 - PRECISION: 0.3289430, PRECISION_RECALL_MIN_DEN: 0.3507929, RECALL: 0.1443527, MAP: 0.2372513, MAP_MIN_DEN: 0.2481392, MRR: 0.6085585, NDCG: 0.3006104, F1: 0.2006518, HIT_RATE: 0.8727634, ARHR_ALL_HITS: 1.1395252, NOVELTY: 0.0237508, AVERAGE_POPULARITY: 0.5261314, DIVERSITY_MEAN_INTER_LIST: 0.9116772, DIVERSITY_HERFINDAHL: 0.9911526, COVERAGE_ITEM: 0.1964976, COVERAGE_ITEM_CORRECT: 0.1529745, COVERAGE_USER: 0.9993377, COVERAGE_USER_CORRECT: 0.8721854, DIVERSITY_GINI: 0.0382841, SHANNON_ENTROPY: 7.5692657, RATIO_DIVERSITY_HERFINDAHL: 0.9919632, RATIO_DIVERSITY_GINI: 0.1094867, RATIO_SHANNON_ENTROPY: 0.7000265, RATIO_AVERAGE_POPULARITY: 

### The metadata.zip file contains details on the search

In [9]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['result_on_test_best', 'time_df', 'result_on_last', 'time_on_last_df', 'time_on_validation_avg', 'cutoff_to_optimize', 'metric_to_optimize', 'time_on_validation_total', 'algorithm_name_recommender', 'hyperparameters_df', 'result_on_validation_best', 'time_on_train_total', 'time_on_train_avg', 'result_on_test_df', 'algorithm_name_search', 'time_on_test_avg', 'hyperparameters_best', 'hyperparameters_best_index', 'result_on_validation_df', 'exception_list', 'time_on_test_total'])

In [10]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

,topK,shrink,similarity,normalize
0,150,314,cosine,True
1,383,664,cosine,False
2,159,547,cosine,True
3,300,1000,cosine,True
4,783,0,cosine,False
5,998,2,cosine,True
6,5,1000,cosine,True
7,693,122,cosine,True
8,513,46,cosine,True
9,5,182,cosine,False


In [11]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.202623,0.228896,0.120448,0.105994,0.117016,0.423969,0.192351,0.151084,0.798141,0.657837,...,0.173835,0.997351,0.796026,0.053959,8.034793,0.994022,0.154324,0.743077,1.774303,0.081837
1,10,0.145435,0.162662,0.080011,0.075451,0.082986,0.354088,0.147499,0.10323,0.678287,0.508359,...,0.017255,0.997351,0.67649,0.005378,4.700364,0.954948,0.015382,0.434701,2.718386,0.075125
2,10,0.203602,0.229579,0.12052,0.107041,0.118379,0.425583,0.194463,0.151413,0.800299,0.662361,...,0.164048,0.997351,0.798179,0.050306,7.938351,0.993629,0.143876,0.734158,1.806573,0.081526
3,10,0.202125,0.228466,0.120046,0.107534,0.119399,0.429007,0.197879,0.15063,0.794655,0.667068,...,0.126964,0.997351,0.79255,0.033902,7.378293,0.990586,0.09696,0.682362,1.980549,0.080017
4,10,0.142098,0.158131,0.076744,0.073414,0.080385,0.345805,0.143196,0.099663,0.663679,0.495413,...,0.012104,0.997351,0.661921,0.005027,4.581923,0.952093,0.014377,0.423747,2.735925,0.07503
5,10,0.193675,0.217827,0.112892,0.102723,0.113711,0.417543,0.186976,0.14264,0.77905,0.644213,...,0.112799,0.997351,0.776987,0.024665,6.889494,0.986169,0.070542,0.637157,2.094322,0.079146
6,10,0.157669,0.17911,0.093214,0.07686,0.084908,0.354809,0.145788,0.117162,0.72842,0.515201,...,0.201133,0.997351,0.72649,0.080293,8.627045,0.996386,0.229641,0.79785,1.564439,0.083728
7,10,0.197825,0.222848,0.115995,0.105119,0.116278,0.423346,0.191634,0.146241,0.786355,0.6555,...,0.124131,0.997351,0.784272,0.02885,7.119046,0.988209,0.082511,0.658386,2.041729,0.07956
8,10,0.20083,0.228163,0.120765,0.105947,0.1177,0.424558,0.194242,0.150831,0.797311,0.660029,...,0.139583,0.997351,0.795199,0.035164,7.40108,0.990311,0.10057,0.684469,1.96181,0.080221


In [12]:
result_best_on_test = search_metadata["result_on_last"]
result_best_on_test

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.328943,0.350793,0.144353,0.237251,0.248139,0.608559,0.30061,0.200652,0.872763,1.139525,...,0.152975,0.999338,0.872185,0.038284,7.569266,0.991963,0.109487,0.700027,1.915292,0.064382


In [13]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'topK': 300, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True}

In [14]:
time_df = search_metadata["time_df"]
time_df

,train,validation,test
0,0.790614,5.419285,6.228587
1,1.550529,8.002338,NaN
2,1.721653,6.326061,6.375081
3,1.495981,7.337181,7.332524
4,1.628996,9.625446,NaN
5,1.810539,9.066864,NaN
6,1.304739,4.667664,NaN
7,1.568717,8.511135,NaN
8,1.85302,7.490336,NaN
9,1.650328,4.953592,NaN


In [15]:
exception_list = search_metadata["exception_list"]
exception_list

[None, None, None, None, None, None, None, None, None, None]

## An example with earlystopping, for FunKSVD

In [16]:
hyperparameters_range_dictionary = {
    "epochs": Categorical([500]),
    "num_factors": Integer(1, 200),
    "sgd_mode": Categorical(["sgd", "adagrad", "adam"]),
    "batch_size": Categorical([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]),
    "item_reg": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
    "user_reg": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
    "learning_rate": Real(low = 1e-4, high = 1e-1, prior = 'log-uniform'),
}

In [17]:
earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [18]:
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = earlystopping_keywargs,     # Additional hyperparameters for the fit function
)

In [19]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = earlystopping_keywargs,     # Additional hyperparameters for the fit function
)

In [20]:
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_FunkSVD_Cython

recommender_class = MatrixFactorization_FunkSVD_Cython

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

In [21]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'epochs': 500, 'num_factors': 53, 'sgd_mode': 'sgd', 'batch_size': 8, 'item_reg': 0.00012885833828867778, 'user_reg': 1.3327983000192677e-05, 'learning_rate': 0.0011498156311265017}
MatrixFactorization_FunkSVD_Cython_Recommender: URM Detected 238 ( 6.1%) items with no interactions.
FUNK_SVD: Processed 640136 (100.0%) in 4.02 sec. MSE loss 1.28E+00. Sample per second: 159340
FUNK_SVD: Epoch 1 of 500. Elapsed time 3.79 sec
FUNK_SVD: Processed 640136 (100.0%) in 3.80 sec. MSE loss 1.17E+00. Sample per second: 168655
FUNK_SVD: Epoch 2 of 500. Elapsed time 7.57 sec
FUNK_SVD: Processed 640136 (100.0%) in 3.67 sec. MSE loss 1.14E+00. Sample per second: 174457
FUNK_SVD: Epoch 3 of 500. Elapsed time 10.45 sec
FUNK_SVD: Processed 640136 (100.0%) in 3.58 sec. MSE loss 1.11E+00. Sample per second: 178704
FUNK_SVD: Epoch 4 of 500. Elapsed time 13.36 sec
FUNK_SVD: Processed 640136 (100.0%) in 3.04 sec

FUNK_SVD: Processed 640136 (100.0%) in 3.25 sec. MSE loss 9.18E-01. Sample per second: 197039
FUNK_SVD: Epoch 27 of 500. Elapsed time 1.57 min
FUNK_SVD: Processed 640136 (100.0%) in 2.77 sec. MSE loss 9.17E-01. Sample per second: 230949
FUNK_SVD: Epoch 28 of 500. Elapsed time 1.61 min
FUNK_SVD: Processed 640136 (100.0%) in 3.35 sec. MSE loss 9.12E-01. Sample per second: 191178
FUNK_SVD: Epoch 29 of 500. Elapsed time 1.65 min
FUNK_SVD: Processed 640136 (100.0%) in 3.27 sec. MSE loss 9.09E-01. Sample per second: 195660
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 6024 (100.0%) in 3.59 sec. Users per second: 1676
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0633798, PRECISION_RECALL_MIN_DEN: 0.0693543, RECALL: 0.0307000, MAP: 0.0248637, MAP_MIN_DEN: 0.0267379, MRR: 0.1524233, NDCG: 0.0606391, F1: 0.0413640, HIT_RATE: 0.4025564, ARHR_ALL_HITS: 0.1929833, NOVELTY: 0.0242611, AVERAGE_POPULARITY: 0.4746087, DIVERSITY_MEAN_INTER_LIST: 0.5401015, DIVERSITY_HERFINDAHL: 0.9540012, COVERAGE_I

FUNK_SVD: Processed 640144 (100.0%) in 5.01 sec. MSE loss 1.46E+00. Sample per second: 127688
FUNK_SVD: Epoch 16 of 500. Elapsed time 1.28 min
FUNK_SVD: Processed 640144 (100.0%) in 4.32 sec. MSE loss 1.40E+00. Sample per second: 148304
FUNK_SVD: Epoch 17 of 500. Elapsed time 1.36 min
FUNK_SVD: Processed 640144 (100.0%) in 4.56 sec. MSE loss 1.34E+00. Sample per second: 140246
FUNK_SVD: Epoch 18 of 500. Elapsed time 1.43 min
FUNK_SVD: Processed 640144 (100.0%) in 4.82 sec. MSE loss 1.30E+00. Sample per second: 132573
FUNK_SVD: Epoch 19 of 500. Elapsed time 1.50 min
FUNK_SVD: Processed 640144 (100.0%) in 5.06 sec. MSE loss 1.26E+00. Sample per second: 126492
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 6024 (100.0%) in 4.83 sec. Users per second: 1248
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.1099104, PRECISION_RECALL_MIN_DEN: 0.1221991, RECALL: 0.0575444, MAP: 0.0504870, MAP_MIN_DEN: 0.0551992, MRR: 0.2610717, NDCG: 0.1046849, F1: 0.0755395, HIT_RATE: 0.5761952, ARHR_ALL_HITS: 

FUNK_SVD: Epoch 44 of 500. Elapsed time 3.66 min
FUNK_SVD: Processed 640144 (100.0%) in 4.63 sec. MSE loss 1.00E+00. Sample per second: 138161
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 6024 (100.0%) in 4.87 sec. Users per second: 1238
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.1073373, PRECISION_RECALL_MIN_DEN: 0.1186354, RECALL: 0.0550678, MAP: 0.0482923, MAP_MIN_DEN: 0.0525795, MRR: 0.2509920, NDCG: 0.1027217, F1: 0.0727912, HIT_RATE: 0.5634130, ARHR_ALL_HITS: 0.3425592, NOVELTY: 0.0224971, AVERAGE_POPULARITY: 0.6962233, DIVERSITY_MEAN_INTER_LIST: 0.5957626, DIVERSITY_HERFINDAHL: 0.9595664, COVERAGE_ITEM: 0.0309039, COVERAGE_ITEM_CORRECT: 0.0218903, COVERAGE_USER: 0.9973510, COVERAGE_USER_CORRECT: 0.5619205, DIVERSITY_GINI: 0.0063850, SHANNON_ENTROPY: 4.9959996, RATIO_DIVERSITY_HERFINDAHL: 0.9603503, RATIO_DIVERSITY_GINI: 0.0182612, RATIO_SHANNON_ENTROPY: 0.4620419, RATIO_AVERAGE_POPULARITY: 2.5205771, RATIO_NOVELTY: 0.0762288, 

FUNK_SVD: Convergence reached! Terminating a

FUNK_SVD: Processed 640256 (100.0%) in 8.26 sec. MSE loss 1.24E+00. Sample per second: 77544
FUNK_SVD: Epoch 16 of 500. Elapsed time 2.65 min
FUNK_SVD: Processed 640256 (100.0%) in 9.38 sec. MSE loss 1.24E+00. Sample per second: 68258
FUNK_SVD: Epoch 17 of 500. Elapsed time 2.80 min
FUNK_SVD: Processed 640256 (100.0%) in 9.75 sec. MSE loss 1.24E+00. Sample per second: 65654
FUNK_SVD: Epoch 18 of 500. Elapsed time 2.96 min
FUNK_SVD: Processed 640256 (100.0%) in 10.05 sec. MSE loss 1.24E+00. Sample per second: 63694
FUNK_SVD: Epoch 19 of 500. Elapsed time 3.12 min
FUNK_SVD: Processed 640256 (100.0%) in 8.93 sec. MSE loss 1.24E+00. Sample per second: 71661
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 6024 (100.0%) in 4.50 sec. Users per second: 1339
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0087317, PRECISION_RECALL_MIN_DEN: 0.0094239, RECALL: 0.0034604, MAP: 0.0027641, MAP_MIN_DEN: 0.0029988, MRR: 0.0251002, NDCG: 0.0057061, F1: 0.0049565, HIT_RATE: 0.0796813, ARHR_ALL_HITS: 0.02

...

FUNK_SVD: Processed 640192 (100.0%) in 0.95 sec. MSE loss 7.94E+00. Sample per second: 674510
FUNK_SVD: Epoch 16 of 500. Elapsed time 19.89 sec
FUNK_SVD: Processed 640192 (100.0%) in 1.26 sec. MSE loss 7.78E+00. Sample per second: 508086
FUNK_SVD: Epoch 17 of 500. Elapsed time 20.20 sec
FUNK_SVD: Processed 640192 (100.0%) in 0.73 sec. MSE loss 7.61E+00. Sample per second: 876447
FUNK_SVD: Epoch 18 of 500. Elapsed time 20.67 sec
FUNK_SVD: Processed 640192 (100.0%) in 1.23 sec. MSE loss 7.45E+00. Sample per second: 521074
FUNK_SVD: Epoch 19 of 500. Elapsed time 21.17 sec
FUNK_SVD: Processed 640192 (100.0%) in 0.57 sec. MSE loss 7.30E+00. Sample per second: 1123081
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 6024 (100.0%) in 4.56 sec. Users per second: 1320
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.1189575, PRECISION_RECALL_MIN_DEN: 0.1325547, RECALL: 0.0621542, MAP: 0.0569873, MAP_MIN_DEN: 0.0625789, MRR: 0.2833098, NDCG: 0.1131901, F1: 0.0816480, HIT_RATE: 0.5966135, ARHR_ALL_H

EvaluatorHoldout: Processed 6036 (100.0%) in 4.82 sec. Users per second: 1253
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'epochs': 15, 'num_factors': 1, 'sgd_mode': 'adagrad', 'batch_size': 64, 'item_reg': 0.01, 'user_reg': 0.01, 'learning_rate': 0.0011332925114552466} - results:
CUTOFF: 10 - PRECISION: 0.1470345, PRECISION_RECALL_MIN_DEN: 0.1571106, RECALL: 0.0618970, MAP: 0.0736408, MAP_MIN_DEN: 0.0774005, MRR: 0.3169586, NDCG: 0.1292467, F1: 0.0871194, HIT_RATE: 0.6504307, ARHR_ALL_HITS: 0.4713302, NOVELTY: 0.0220970, AVERAGE_POPULARITY: 0.7643492, DIVERSITY_MEAN_INTER_LIST: 0.4658630, DIVERSITY_HERFINDAHL: 0.9465786, COVERAGE_ITEM: 0.0121040, COVERAGE_ITEM_CORRECT: 0.0108164, COVERAGE_USER: 0.9993377, COVERAGE_USER_CORRECT: 0.6500000, DIVERSITY_GINI: 0.0045915, SHANNON_ENTROPY: 4.4184519, RATIO_DIVERSITY_HERFINDAHL: 0.9473519, RATIO_DIVERSITY_GINI: 0.0131319, RATIO_SHANNON_ENTROPY: 0.4086289, RATIO_AVERAGE_POPULARITY: 2.7672173, RATIO_NOVELTY: 0.0748730, 



FUNK_SVD: Processed 640135 (100.0%) in 0.40 sec. MSE loss 1.13E+00. Sample per second: 1619413
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 6024 (100.0%) in 4.61 sec. Users per second: 1306
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.1102922, PRECISION_RECALL_MIN_DEN: 0.1217955, RECALL: 0.0561448, MAP: 0.0518545, MAP_MIN_DEN: 0.0568572, MRR: 0.2680191, NDCG: 0.1090101, F1: 0.0744105, HIT_RATE: 0.5678951, ARHR_ALL_HITS: 0.3659392, NOVELTY: 0.0222883, AVERAGE_POPULARITY: 0.7294135, DIVERSITY_MEAN_INTER_LIST: 0.3999889, DIVERSITY_HERFINDAHL: 0.9399922, COVERAGE_ITEM: 0.0121040, COVERAGE_ITEM_CORRECT: 0.0110739, COVERAGE_USER: 0.9973510, COVERAGE_USER_CORRECT: 0.5663907, DIVERSITY_GINI: 0.0041905, SHANNON_ENTROPY: 4.2718474, RATIO_DIVERSITY_HERFINDAHL: 0.9407602, RATIO_DIVERSITY_GINI: 0.0119851, RATIO_SHANNON_ENTROPY: 0.3950706, RATIO_AVERAGE_POPULARITY: 2.6407375, RATIO_NOVELTY: 0.0755212, 

FUNK_SVD: Epoch 25 of 500. Elapsed time 34.54 sec
FUNK_SVD: Processed 640135 (100.0%) in 0.3

FUNK_SVD: Processed 640135 (100.0%) in 1.00 sec. MSE loss 1.28E+00. Sample per second: 637050
FUNK_SVD: Epoch 16 of 500. Elapsed time 15.27 sec
FUNK_SVD: Processed 640135 (100.0%) in 0.46 sec. MSE loss 1.26E+00. Sample per second: 1399274
FUNK_SVD: Epoch 17 of 500. Elapsed time 15.72 sec
FUNK_SVD: Processed 640135 (100.0%) in 0.69 sec. MSE loss 1.24E+00. Sample per second: 923494
FUNK_SVD: Epoch 18 of 500. Elapsed time 15.96 sec
FUNK_SVD: Processed 640135 (100.0%) in 0.91 sec. MSE loss 1.23E+00. Sample per second: 702973
FUNK_SVD: Epoch 19 of 500. Elapsed time 16.18 sec
FUNK_SVD: Processed 640135 (100.0%) in 1.13 sec. MSE loss 1.21E+00. Sample per second: 568212
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 6024 (100.0%) in 3.43 sec. Users per second: 1756
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.1141102, PRECISION_RECALL_MIN_DEN: 0.1266480, RECALL: 0.0590362, MAP: 0.0535163, MAP_MIN_DEN: 0.0586453, MRR: 0.2690154, NDCG: 0.1102873, F1: 0.0778143, HIT_RATE: 0.5808433, ARHR_ALL_H

FUNK_SVD: Processed 800192 (100.0%) in 0.58 sec. MSE loss 7.71E+00. Sample per second: 1375515
FUNK_SVD: Epoch 14 of 15. Elapsed time 4.51 sec
FUNK_SVD: Processed 800192 (100.0%) in 0.82 sec. MSE loss 7.51E+00. Sample per second: 972444
FUNK_SVD: Epoch 15 of 15. Elapsed time 4.75 sec
FUNK_SVD: Terminating at epoch 15. Elapsed time 4.75 sec
EvaluatorHoldout: Processed 6036 (100.0%) in 3.44 sec. Users per second: 1754
SearchBayesianSkopt: Best config evaluated with evaluator_test with constructor data for final test. Config: {'epochs': 15, 'num_factors': 1, 'sgd_mode': 'adagrad', 'batch_size': 64, 'item_reg': 0.01, 'user_reg': 0.01, 'learning_rate': 0.0011332925114552466} - results:
CUTOFF: 10 - PRECISION: 0.1763419, PRECISION_RECALL_MIN_DEN: 0.1864401, RECALL: 0.0674992, MAP: 0.1051057, MAP_MIN_DEN: 0.1090986, MRR: 0.3674129, NDCG: 0.1543828, F1: 0.0976286, HIT_RATE: 0.6706428, ARHR_ALL_HITS: 0.5930055, NOVELTY: 0.0221507, AVERAGE_POPULARITY: 0.7488238, DIVERSITY_MEAN_INTER_LIST: 0.5295

In [22]:
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

,epochs,num_factors,sgd_mode,batch_size,item_reg,user_reg,learning_rate
0,5,53,sgd,8,0.000129,0.000013,0.00115
1,20,40,adagrad,16,0.000336,0.000021,0.002356
2,500,122,sgd,256,0.006262,0.000235,0.000167
3,10,48,sgd,2,0.000157,0.000026,0.008652
4,5,132,adagrad,32,0.000016,0.001534,0.090911
5,120,108,adagrad,32,0.00358,0.000014,0.001229
6,260,20,adagrad,4,0.000015,0.002114,0.000453
7,15,1,adagrad,64,0.01,0.01,0.001133
8,5,1,adagrad,1,0.00001,0.00001,0.001335
9,5,1,adagrad,1,0.01,0.00001,0.00089


In [23]:
time_df = search_metadata["time_df"]
time_df

,train,validation,test
0,105.788402,3.551863,3.345683
1,228.568995,4.646658,4.720899
2,4691.106388,4.741997,NaN
3,157.704445,4.618278,NaN
4,370.884716,4.37645,NaN
5,1548.496231,4.912879,NaN
6,900.066335,4.686435,4.761179
7,51.814222,4.571804,4.823095
8,41.532243,4.548487,NaN
9,28.977696,3.262919,NaN


In [24]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'epochs': 15,
 'num_factors': 1,
 'sgd_mode': 'adagrad',
 'batch_size': 64,
 'item_reg': 0.01,
 'user_reg': 0.01,
 'learning_rate': 0.0011332925114552466}

## How to use the predefined hyperparameter ranges

Function runParameterSearch_Collaborative takes as input all needed to optimize any of the recommenders in the framework as well as other parameters such as which similarity heuristics to use for the KNNs, whether to parallelize the training of KNNs and so on...

In [25]:
similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"]

In [26]:
from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative, runHyperparameterSearch_Content
from Recommenders.NonPersonalizedRecommender import TopPop, Random
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

recommender_class = TopPop

runHyperparameterSearch_Collaborative(recommender_class,
       URM_train = URM_train,
       URM_train_last_test = URM_train_validation,
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)

SearchSingleCase: Resuming 'TopPopRecommender' Failed, no such file exists.

SearchSingleCase: Testing config: {}
TopPopRecommender: URM Detected 238 ( 6.1%) items with no interactions.
EvaluatorHoldout: Processed 6024 (100.0%) in 2.63 sec. Users per second: 2291
SearchSingleCase: New best config found. Config 0: {} - results: PRECISION: 0.1217961, PRECISION_RECALL_MIN_DEN: 0.1349623, RECALL: 0.0622819, MAP: 0.0596391, MAP_MIN_DEN: 0.0653490, MRR: 0.2875275, NDCG: 0.1124584, F1: 0.0824182, HIT_RATE: 0.5946215, ARHR_ALL_HITS: 0.4067459, NOVELTY: 0.0220221, AVERAGE_POPULARITY: 0.7784131, DIVERSITY_MEAN_INTER_LIST: 0.4049413, DIVERSITY_HERFINDAHL: 0.9404874, COVERAGE_ITEM: 0.0113314, COVERAGE_ITEM_CORRECT: 0.0100438, COVERAGE_USER: 0.9973510, COVERAGE_USER_CORRECT: 0.5930464, DIVERSITY_GINI: 0.0042281, SHANNON_ENTROPY: 4.2698400, RATIO_DIVERSITY_HERFINDAHL: 0.9412558, RATIO_DIVERSITY_GINI: 0.0120924, RATIO_SHANNON_ENTROPY: 0.3948849, RATIO_AVERAGE_POPULARITY: 2.8181335, RATIO_NOVELTY: 0.0

### The process is similar for content-based algorithms.
* Add the ICM data and its name (it will be used for file names)
* Remove the evaluator for earlystopping

In [27]:
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

recommender_class = ItemKNNCBFRecommender

runHyperparameterSearch_Content(recommender_class,
       URM_train = URM_train,
       ICM_object = ICM_all,
       ICM_name = "ICM_genres",
       URM_train_last_test = URM_train_validation,
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)

SearchBayesianSkopt: Resuming 'ItemKNNCBFRecommender_ICM_genres_cosine' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 454, 'shrink': 169, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'BM25'}
ItemKNNCBFRecommender: URM Detected 238 ( 6.1%) items with no interactions.
SearchBayesianSkopt: Resuming 'ItemKNNCBFRecommender_ICM_genres_jaccard' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 927, 'shrink': 987, 'similarity': 'jaccard', 'normalize': True}
ItemKNNCBFRecommender: URM Detected 238 ( 6.1%) items with no interactions.
SearchBayesianSkopt: Resuming 'ItemKNNCBFRecommender_ICM_genres_dice' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 750, 'shrink': 926, 'similarity': 'dice', 'normalize': True}
It


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 6.8641
Function value obtained: -0.0218
Current minimum: -0.0221
Iteration No: 3 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 6, 'shrink': 608, 'similarity': 'dice', 'normalize': False}
ItemKNNCBFRecommender: URM Detected 238 ( 6.1%) items with no interactions.
Similarity column 3883 (100.0%), 20947.92 column/sec. Elapsed time 0.19 sec
EvaluatorHoldout: Processed 6036 (100.0%) in 5.33 sec. Users per second: 1132
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'topK': 347, 'shrink': 379, 'similarity': 'tversky', 'normalize': True, 'tversky_alpha': 0.8564596806756939, 'tversky_beta': 0.09091909290431067} - results:
CUTOFF: 10 - PRECISION: 0.0723492, PRECISION_RECALL_MIN_DEN: 0.0751475, RECALL: 0.0251246, MAP: 0.0311188, MAP_MIN_DEN: 0.0319933, MRR: 0.1671991, NDCG: 0.0438749, F1: 0.0372971, HIT_RATE: 0.4037442, ARHR_ALL_HITS: 0.2239745, NOVELTY: 0.028286


Similarity column 3883 (100.0%), 14454.75 column/sec. Elapsed time 0.27 sec
EvaluatorHoldout: Processed 6024 (100.0%) in 3.82 sec. Users per second: 1576
SearchBayesianSkopt: Config 3 is suboptimal. Config: {'topK': 5, 'shrink': 1000, 'similarity': 'jaccard', 'normalize': True} - results: PRECISION: 0.0497178, PRECISION_RECALL_MIN_DEN: 0.0540287, RECALL: 0.0230467, MAP: 0.0199906, MAP_MIN_DEN: 0.0214813, MRR: 0.1305180, NDCG: 0.0347948, F1: 0.0314943, HIT_RATE: 0.3343293, ARHR_ALL_HITS: 0.1601152, NOVELTY: 0.0292292, AVERAGE_POPULARITY: 0.1910602, DIVERSITY_MEAN_INTER_LIST: 0.9525647, DIVERSITY_HERFINDAHL: 0.9952407, COVERAGE_ITEM: 0.3757404, COVERAGE_ITEM_CORRECT: 0.1079063, COVERAGE_USER: 0.9973510, COVERAGE_USER_CORRECT: 0.3334437, DIVERSITY_GINI: 0.0765684, SHANNON_ENTROPY: 8.5402188, RATIO_DIVERSITY_HERFINDAHL: 0.9960537, RATIO_DIVERSITY_GINI: 0.2189882, RATIO_SHANNON_ENTROPY: 0.7898197, RATIO_AVERAGE_POPULARITY: 0.6917064, RATIO_NOVELTY: 0.0990397, 

Iteration No: 4 ended. Searc


Iteration No: 5 ended. Search finished for the next optimal point.
Time taken: 4.8693
Function value obtained: -0.0231
Current minimum: -0.0242
Iteration No: 6 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 365, 'shrink': 0, 'similarity': 'tversky', 'normalize': True, 'tversky_alpha': 0.9451994341143322, 'tversky_beta': 1.9034035449301236}
ItemKNNCBFRecommender: URM Detected 238 ( 6.1%) items with no interactions.
EvaluatorHoldout: Processed 6024 (100.0%) in 5.04 sec. Users per second: 1195
SearchBayesianSkopt: Config 3 is suboptimal. Config: {'topK': 236, 'shrink': 897, 'similarity': 'cosine', 'normalize': False, 'feature_weighting': 'TF-IDF'} - results: PRECISION: 0.0589475, PRECISION_RECALL_MIN_DEN: 0.0637218, RECALL: 0.0265087, MAP: 0.0242811, MAP_MIN_DEN: 0.0258535, MRR: 0.1472971, NDCG: 0.0402735, F1: 0.0365713, HIT_RATE: 0.3670319, ARHR_ALL_HITS: 0.1871194, NOVELTY: 0.0286932, AVERAGE_POPULARITY: 0.2084592, DIVERSITY_MEAN_INTER_LIST

Similarity column 3883 (100.0%), 8014.72 column/sec. Elapsed time 0.48 sec
EvaluatorHoldout: Processed 6024 (100.0%) in 5.49 sec. Users per second: 1098
SearchBayesianSkopt: Config 6 is suboptimal. Config: {'topK': 569, 'shrink': 999, 'similarity': 'jaccard', 'normalize': True} - results: PRECISION: 0.0538679, PRECISION_RECALL_MIN_DEN: 0.0583835, RECALL: 0.0247511, MAP: 0.0215132, MAP_MIN_DEN: 0.0229863, MRR: 0.1305522, NDCG: 0.0362937, F1: 0.0339177, HIT_RATE: 0.3389774, ARHR_ALL_HITS: 0.1661128, NOVELTY: 0.0290451, AVERAGE_POPULARITY: 0.1962369, DIVERSITY_MEAN_INTER_LIST: 0.8982177, DIVERSITY_HERFINDAHL: 0.9898069, COVERAGE_ITEM: 0.2547000, COVERAGE_ITEM_CORRECT: 0.0767448, COVERAGE_USER: 0.9973510, COVERAGE_USER_CORRECT: 0.3380795, DIVERSITY_GINI: 0.0385445, SHANNON_ENTROPY: 7.5063410, RATIO_DIVERSITY_HERFINDAHL: 0.9906155, RATIO_DIVERSITY_GINI: 0.1102384, RATIO_SHANNON_ENTROPY: 0.6942042, RATIO_AVERAGE_POPULARITY: 0.7104476, RATIO_NOVELTY: 0.0984159, 

Similarity column 3883 (100.0

SearchBayesianSkopt: Testing config: {'topK': 5, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'BM25'}
ItemKNNCBFRecommender: URM Detected 238 ( 6.1%) items with no interactions.
Iteration No: 9 ended. Search finished for the next optimal point.
Time taken: 7.1110
Function value obtained: -0.0230
Current minimum: -0.0256
Iteration No: 10 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 32, 'shrink': 0, 'similarity': 'dice', 'normalize': True}
ItemKNNCBFRecommender: URM Detected 238 ( 6.1%) items with no interactions.
Similarity column 3883 (100.0%), 16843.04 column/sec. Elapsed time 0.23 sec
Similarity column 3883 (100.0%), 12634.34 column/sec. Elapsed time 0.31 sec
EvaluatorHoldout: Processed 6024 (100.0%) in 6.95 sec. Users per second: 867
SearchBayesianSkopt: Config 9 is suboptimal. Config: {'topK': 763, 'shrink': 885, 'similarity': 'jaccard', 'normalize': True} - results: PRECISION: 0.0548473, PRECISION_R


SearchBayesianSkopt: Saving model in result_experiments/ItemKNNCBFRecommender_ICM_genres_dice

ItemKNNCBFRecommender: Saving model in file 'result_experiments/ItemKNNCBFRecommender_ICM_genres_dice_best_model_last'
EvaluatorHoldout: Processed 6036 (100.0%) in 5.92 sec. Users per second: 1019
SearchBayesianSkopt: Best config evaluated with evaluator_test with constructor data for final test. Config: {'topK': 355, 'shrink': 176, 'similarity': 'cosine', 'normalize': False, 'feature_weighting': 'TF-IDF'} - results:
CUTOFF: 10 - PRECISION: 0.0870278, PRECISION_RECALL_MIN_DEN: 0.0900327, RECALL: 0.0293889, MAP: 0.0410426, MAP_MIN_DEN: 0.0421953, MRR: 0.2014210, NDCG: 0.0550319, F1: 0.0439396, HIT_RATE: 0.4541087, ARHR_ALL_HITS: 0.2806119, NOVELTY: 0.0286268, AVERAGE_POPULARITY: 0.2092813, DIVERSITY_MEAN_INTER_LIST: 0.8945452, DIVERSITY_HERFINDAHL: 0.9894397, COVERAGE_ITEM: 0.1820757, COVERAGE_ITEM_CORRECT: 0.0875612, COVERAGE_USER: 0.9993377, COVERAGE_USER_CORRECT: 0.4538079, DIVERSITY_GINI:

### You can use the "partial" function to pre-set all attributes that are needed and use it to loop on the recommender_classes

In [28]:
from Recommenders.NonPersonalizedRecommender import TopPop, Random
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
import os, multiprocessing
from functools import partial


runHyperparameterSearch_Collaborative_partial = partial(runHyperparameterSearch_Collaborative,
       URM_train = URM_train,
       URM_train_last_test = URM_train_validation,
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)


In [29]:
collaborative_algorithm_list = [
    Random,
    TopPop,
    P3alphaRecommender,
]

### You can call it as part of a parallel pool, hence parallizing the optimization of the various models

In [30]:
pool = multiprocessing.Pool(processes=int(multiprocessing.cpu_count()), maxtasksperchild=1)
pool.map(runHyperparameterSearch_Collaborative_partial, collaborative_algorithm_list)

SearchSingleCase: Resuming 'RandomRecommender' Failed, no such file exists.

SearchSingleCase: Testing config: {}
RandomRecommender: URM Detected 238 ( 6.1%) items with no interactions.
SearchBayesianSkopt: Resuming 'P3alphaRecommender' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 593, 'alpha': 1.5162625884190866, 'normalize_similarity': False}
P3alphaRecommender: URM Detected 238 ( 6.1%) items with no interactions.
SearchSingleCase: Resuming 'TopPopRecommender'... Loaded 1 configurations.
TopPopRecommender: URM Detected 212 ( 5.5%) items with no interactions.
SearchSingleCase: Resuming 'TopPopRecommender'... Result on last already available.
EvaluatorHoldout: Processed 6024 (100.0%) in 4.76 sec. Users per second: 1266
SearchSingleCase: New best config found. Config 0: {} - results: PRECISION: 0.0072709, PRECISION_RECALL_MIN_DEN: 0.0077562, RECALL: 0.0028381, MAP: 0.0022469, MAP_MIN_DEN: 0.0024


P3alphaRecommender: Saving model in file 'result_experiments/P3alphaRecommender_best_model'
P3alphaRecommender: Saving complete
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 15.3414
Function value obtained: -0.1040
Current minimum: -0.1040
Iteration No: 3 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 18, 'alpha': 1.7608333614795886, 'normalize_similarity': False}
P3alphaRecommender: URM Detected 238 ( 6.1%) items with no interactions.
EvaluatorHoldout: Processed 6024 (100.0%) in 4.06 sec. Users per second: 1484
SearchBayesianSkopt: Config 2 is suboptimal. Config: {'topK': 18, 'alpha': 1.7608333614795886, 'normalize_similarity': False} - results: PRECISION: 0.0425299, PRECISION_RECALL_MIN_DEN: 0.0468866, RECALL: 0.0216756, MAP: 0.0173111, MAP_MIN_DEN: 0.0192689, MRR: 0.1231091, NDCG: 0.0325182, F1: 0.0287160, HIT_RATE: 0.3014608, ARHR_ALL_HITS: 0.1451413, NOVELTY: 0.0303234, AVERAGE_POPULARITY: 0.1446706, DIVERSITY_MEA


Iteration No: 8 ended. Search finished for the next optimal point.
Time taken: 10.3573
Function value obtained: -0.1004
Current minimum: -0.1078
Iteration No: 9 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 5, 'alpha': 1.0204916543650424, 'normalize_similarity': True}
P3alphaRecommender: URM Detected 238 ( 6.1%) items with no interactions.
EvaluatorHoldout: Processed 6024 (100.0%) in 4.43 sec. Users per second: 1361
SearchBayesianSkopt: Config 8 is suboptimal. Config: {'topK': 5, 'alpha': 1.0204916543650424, 'normalize_similarity': True} - results: PRECISION: 0.0985060, PRECISION_RECALL_MIN_DEN: 0.1026572, RECALL: 0.0372631, MAP: 0.0555751, MAP_MIN_DEN: 0.0586253, MRR: 0.2952565, NDCG: 0.0967650, F1: 0.0540718, HIT_RATE: 0.4973440, ARHR_ALL_HITS: 0.3977612, NOVELTY: 0.0313592, AVERAGE_POPULARITY: 0.2009188, DIVERSITY_MEAN_INTER_LIST: 0.8220619, DIVERSITY_HERFINDAHL: 0.9821925, COVERAGE_ITEM: 0.2106619, COVERAGE_ITEM_CORRECT: 0.1197528, CO

[None, None, None]

### Or sequentially, note that since resume_from_saved = True this new call to the function will just load the optimized values and not re-run it 

In [31]:
for recommender_class in collaborative_algorithm_list:

    try:

        runHyperparameterSearch_Collaborative_partial(recommender_class)

    except Exception as e:

        print("On recommender {} Exception {}".format(recommender_class, str(e)))
        traceback.print_exc()

SearchSingleCase: Resuming 'RandomRecommender'... Loaded 1 configurations.
RandomRecommender: URM Detected 212 ( 5.5%) items with no interactions.
SearchSingleCase: Resuming 'RandomRecommender'... Result on last already available.
SearchSingleCase: Resuming 'TopPopRecommender'... Loaded 1 configurations.
TopPopRecommender: URM Detected 212 ( 5.5%) items with no interactions.
SearchSingleCase: Resuming 'TopPopRecommender'... Result on last already available.
SearchBayesianSkopt: Resuming 'P3alphaRecommender'... Loaded 10 configurations.
P3alphaRecommender: URM Detected 212 ( 5.5%) items with no interactions.
SearchBayesianSkopt: Resuming 'P3alphaRecommender'... Result on last already available.


## How to load and save a model
### The models are saved in two files:
* RECOMMENDER_NAME + "_best_model.zip": contains the model trained on the URM_train
* RECOMMENDER_NAME + "_best_model_last.zip": contains the model trained on the URM_train_validation

#### You can load them using the "load_model" function of the Recommender class, to do so:
* Create an instance of the recommender you want to load and provide the needed URM data and ICM data, if needed. The URM you pass is the one that will be used as user profiles to generate the recommendations.
* Given the instance, call the load_model function
* You can now use the recommender to generate recommendations.

#### If you want to save your own models use the "save_model" function, it takes thesame parameters as "load_model"

In [32]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

recommender_object = P3alphaRecommender(URM_all)
recommender_object

P3alphaRecommender: URM Detected 177 ( 4.6%) items with no interactions.


In [33]:
recommender_object.load_model(output_folder_path, 
                              file_name = recommender_object.RECOMMENDER_NAME + "_best_model_last.zip" )

P3alphaRecommender: Loading model from file 'result_experiments/P3alphaRecommender_best_model_last.zip'
P3alphaRecommender: Loading complete


In [34]:
recommender_object.W_sparse

<3883x3883 sparse matrix of type '<class 'numpy.float32'>'
	with 221647 stored elements in Compressed Sparse Row format>

In [35]:
user_id = 10
recommender_object.recommend(user_id, cutoff = 10)

[48, 244, 23, 22, 40, 335, 246, 537, 350, 515]

In [36]:
recommender_object.save_model(output_folder_path, 
                              file_name = recommender_object.RECOMMENDER_NAME + "_my_own_save.zip" )

P3alphaRecommender: Saving model in file 'result_experiments/P3alphaRecommender_my_own_save.zip'
P3alphaRecommender: Saving complete
